# Quantity

## Getting Started

Before we start, let's import the symbols we're going to use from `fv3gfs.util`, as well as `numpy` and `gt4py` to allocate data.

In [ ]:
from fv3gfs.util import Quantity, X_DIM, Y_DIM, Z_DIM, X_INTERFACE_DIM, Y_INTERFACE_DIM, Z_INTERFACE_DIM
import numpy as np
import gt4py

## A Quick Look

Let's check out some of the features we just went over in slides. We're going to initialize a quantity a couple different ways, check out how the `quantity.storage` and `quantity.data` attributes relate to each other, and do a couple indexing operations.

### Initialization

First, we'll show how Quantity can be initialized from a numpy array. The initialization routine has three required arguments (data, dims, and units), and three optional arguments (origin, extent, and gt4py_backend).

In [ ]:
help(Quantity.__init__)

This cell shows that Quantity does not require `gt4py_backend`, but if you do you won't be able to access its `.storage` attribute. In this workshop, be sure to supply a backend.

In [ ]:
array = np.zeros([6, 6])
quantity = Quantity(
    array,
    origin=(1, 1),
    extent=(4, 4),
    dims=[X_DIM, Y_DIM],
    units="degK"
    # we're not passing in gt4py_backend
)

print(quantity.storage)  # this line will trigger a TypeError

Above we imported some symbols for dimension types. Quantity requires these symbols to be used in the `dims` argument for dimensionally-aware features to be functional. For example, the halo updates use those particular values to determine the orientation of the array when copying halo regions. This is an example of self-documenting code, where the documentation of quantity dimensions is directly used to perform halo updates.

Also a quick note that `units` is a required input. Technically you can insert an empty string `""` for units and the code wil run. If you're going to do this instead of inserting unit information, we suggest at least using `"unknown"` or a similar string, since an empty string is a unit (unitless).

Keep in mind when deciding whether to write units that for less time than you need to take a sip of your morning coffee, you could save a grad student (or your future self) weeks of debugging their code.

### View

Now that we can create a Quantity, we can use it for indexing. Quantity enables you to write code which is halo-aware without writing the number of halo points, because Quantity already has that information.

The simplest and possibly most common example is to retrieve the entire compute domain with no halos. Quantity provides a convenient way to interact with and index on the compute domain for an array which contains halos.

In [ ]:
quantity.view[:] = np.arange(16).reshape([4, 4])
print(quantity.view[:])
print(type(quantity.view))
print(type(quantity.view[:]))

Note that `view` itself is not a numpy array. It is a custom object which provides indexing operations, and indexing returns a numpy array.

If we look at the entire array, we can see the halos are left untouched.

In [ ]:
print(quantity.data)

An index or slice of view can also be used to modify the compute domain. In the cell below, set the compute domain values to 1.

In [ ]:
# Initialize data to 0 to reset any earlier changes
quantity.data[:] = 0.

# Exercise: set the compute domain values to 1.

# Solution
quantity.view[:] = 1.

print(quantity.data)
# expected:
#[[0. 0. 0. 0. 0. 0.]
# [0. 1. 1. 1. 1. 0.]
# [0. 1. 1. 1. 1. 0.]
# [0. 1. 1. 1. 1. 0.]
# [0. 1. 1. 1. 1. 0.]
# [0. 0. 0. 0. 0. 0.]]

### Storage integration

Quantity has integration both to initialize a Quantity from a GT4py storage object, and to retrieve a storage from the Quantity. The ndarray attribute `quantity.data` shares the same memory as the storage attribute `quantity.storage`, so that modifying one will modify both without performing a copy.

This first cell shows how a Quantity can be used to initialize a storage.

In [ ]:
array = np.zeros([6, 6])
quantity = Quantity(
    array,
    origin=(1, 1),
    extent=(4, 4),
    dims=[X_DIM, Y_DIM],
    units="degK",
    gt4py_backend="numpy",
)
print(quantity.storage)

The `data` attribute and view slices provide a numpy (or if using a GPU storage, cupy) array for the underlying data. Modifying any of these will modify all three.

In [ ]:
print(type(quantity.storage))
print(type(quantity.data))

In the cell below, modify some index values of `quantity.storage` and see how it changes `quantity.data` and `quantity.view`.

In [ ]:
# Solution
quantity.storage[0, 0] = 1
quantity.storage[1, 1] = 2
quantity.storage[4, 4] = 3
print(quantity.data)
print(quantity.view[:])

## Moving Forward

Let's do some more indexing on `view`, past looking at the entire compute domain.

In [ ]:
array = np.zeros([6, 6])
quantity = Quantity(
    array,
    origin=(1, 1),
    extent=(4, 4),
    dims=[X_DIM, Y_DIM],
    units="degK",
    gt4py_backend="numpy",
)
quantity.view[:] = np.arange(16).reshape([4, 4])
print(quantity.data)

Providing indices in `view` has fairly complex behavior, to deal with the complexities of having halos. If you need to get more "normal" indexing, you can first create a numpy array with `view[:]`, and then provide indexing on the "normal" numpy array. For example, -1 does not behave the same as for a numpy array or list, because you may actually want to index into the halo instead of wrapping around to the other edge of the compute domain:

In [ ]:
# gives the second through second-last points on each axis
print(quantity.view[:][1:-1, 1:-1])

In [ ]:
# -1 is treated as the halo point before the origin
# since end is before start, gives an empty view
print(quantity.view[1:-1, 1:-1])
# This line is equivalent to the above line
print(quantity.data[quantity.origin[0] + 1:quantity.origin[0] - 1, quantity.origin[0] + 1:quantity.origin[0] + 1])

The exact behavior of `quantity.view` and its attributes (explored further in "moving forward") is evolving as we use it more and collect feedback on its use cases. If you have feedback or use cases you'd like to bring to our attention, please contact us about it!

In [ ]:
print(help(quantity.view))

When you use these view objects, you can think of the start and end indices as being offset from some reference index. This means that a negative index refers to a location before the reference, and not necessarily to the end of the array.

For one attribute (`quantity.view.interior`), the reference is different for the start and end indices.

The behavior of these views may seem non-intuitive to someone familiar with numpy array indexing. For simple analysis code running in the compute domain, these view attributes are not needed. The use case for these views is to access either halo or compute data near corners and edges of the compute domain, and the two "weird" behaviors (negatives and the behavior of 0:0 for `quantity.view.interior`) are designed for that use case.

Let's start by creating a quantity with increasing values, so we can tell what regions we're viewing. We'll also use a quantity with two halo points.

In [ ]:
# set quantity.data to sequential values so we can see what region we're viewing
array = np.zeros([7, 7])
quantity = Quantity(
    np.arange(49).reshape([7, 7]),
    origin=(2, 2),
    extent=(3, 3),
    dims=[X_DIM, Y_DIM],
    units="degK",
    gt4py_backend="numpy",
)
print(quantity.storage)

In [ ]:
quantity.view.southwest[:]

The closest thing to what we've already seen is `quantity.view.southwest`. This one is actually identical to `quantity.view`, and is provided for completeness.

In [ ]:
print(quantity.view.southwest[:2, :2])
print(quantity.view[:2, :2])

Other corners are useful for accessing data in the neighborhood of those corners. For example, this will access the compute data immediately next to the northeast corner:

In [ ]:
quantity.view.northeast[-2:, -2:]

while the following will access the halo data next to the northeast corner:

In [ ]:
# 0 and 2 are both offsets from the northeast corner
quantity.view.northeast[0:2, 0:2]

Exercise: use `quantity.view.southeast` to print the halo points on the east edge.

In [ ]:
# Solution
print(quantity.view.southeast[0, :])

`quantity.view.interior` may appear awkward, when you see what it returns for this slice:

In [ ]:
print(quantity.view.interior[0:0, 0:0])

The use case for `quantity.view.interior` is to make small modifications on the boundaries that mark the compute domain (called "interior" because compute is an overloaded term).

Exercise: Below, use `quantity.view.interior` to print the quantity including the first ring of halo points but excluding the second ring. The output should look identical to the `data` operation below:

In [ ]:
# Solution
print(quantity.view.interior[-1:1, -1:1])

In [ ]:
print(quantity.data[
    quantity.origin[0] - 1:quantity.origin[0] + quantity.extent[0] + 1,
    quantity.origin[1] - 1:quantity.origin[1] + quantity.extent[1] + 1
])

You can see from this example that the amount of boilerplate needed to use `quantity.data` is pretty high, if you don't want your code to break when the size of the domain or halo changes. Once you know what `quantity.view.interior` is doing, it's much easier to read and write than `quantity.data`.

The help routine can give useful information about the attributes and methods available on an object. We encourage you to check out the documentation if you need to remember the name or purpose of a method or attribute.

In [ ]:
help(Quantity)

In [ ]:
help(quantity.view)